In [15]:
import os
import sys
sys.path.append('/kaggle/input/timm-0-6-9/pytorch-image-models-master')
import glob
import numpy as np
import pandas as pd
import random
import math
import gc
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef
from helper import *
tqdm.pandas()
from albumentations.pytorch import ToTensorV2
import albumentations as A
import timm
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch import nn
import torch
from sklearn.preprocessing import OneHotEncoder
from multiprocessing import Pool
import random
import cv2
from math import ceil
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
tqdm.pandas()
from glob import glob
print('Done 1')

Done 1


In [7]:
%%time
helmet_data= pd.read_csv('./test_baseline_helmets.csv')
tracking_data = pd.read_csv('./test_player_tracking.csv')
labels = pd.read_csv('./sample_submission.csv')

CPU times: user 37.8 ms, sys: 4.58 ms, total: 42.4 ms
Wall time: 42.2 ms


In [8]:
%%time
label_exp = expand_contact_id(labels)
use_cols = [
    'x_position', 'y_position', 'speed', 'distance',
    'direction', 'orientation', 'acceleration', 'sa',
    'team', 'position'
]
combined_df = create_features(label_exp, tracking_data, use_cols=use_cols)

CPU times: user 420 ms, sys: 99.8 ms, total: 520 ms
Wall time: 520 ms


In [9]:
def add_bboxes_details(frame, player1, player2, game_play, view):
    '''
    takes each row in comined_df and adds the required bbox information
    (may need more processing to convert to correct datatype)
    '''
    bboxes = []
    window = 24
    query = f"frame >= {frame-window}  and frame <= {frame+window} and game_play == '{game_play}'"
    if player2 == 'G':
        query += f"  and nfl_player_id in ({player1}, 'G')"
    else:
        query += f"  and nfl_player_id in ({player1}, {player2})"
    filt = helmet_data.query(query)

    tmp = filt[filt["view"] == view]
    tmp = tmp.groupby('frame')[['left','width','top','height']].mean()
    tmp['centre_x'] = tmp['left'] + tmp['width']/2
    tmp['centre_y'] = tmp['top'] + tmp['height']/2
    frame_list = tmp.index # these are the available frames
    for fr in range(frame - window, frame + window + 1):
        if fr in frame_list:
            x, y = tmp.loc[fr][['centre_x','centre_y']]
            bboxes.append([x, y])
        else:
            bboxes.append([np.nan, np.nan])
    
    bboxes = pd.DataFrame(np.array(bboxes)).interpolate(limit_direction='both').values
    return np.array(bboxes)
   

In [12]:
combined_df['bbox_endzone'] = combined_df.progress_apply(
lambda x: add_bboxes_details(x['frame'],x['nfl_player_id_1'],x['nfl_player_id_2'],x['game_play'], 'Endzone'),
axis=1
    )
combined_df['bbox_sideline'] = combined_df.progress_apply(
lambda x: add_bboxes_details(x['frame'],x['nfl_player_id_1'],x['nfl_player_id_2'],x['game_play'], 'Sideline'),
axis=1
    )

100%|██████████| 6631/6631 [01:12<00:00, 91.94it/s] 


In [13]:
def read_image(path, cx, cy, view, aug):
    img_new = np.zeros((1, 256, 256), dtype=np.float32)
    if os.path.isfile(path):
        if view == 'Endzone':
            img = cv2.imread(path, 0)[
                cy-76:cy+180, cx-128:cx+128].copy()
            img_new[0, :img.shape[0], :img.shape[1]] = img
        else:
            img = cv2.imread(path, 0)[
                cy-128:cy+128, cx-128:cx+128].copy()
            img_new[0, :img.shape[0], :img.shape[1]] = img

    return aug(image=img_new.transpose(1, 2, 0))['image'][0]

In [ ]:
CFG = {
    'seed': 1,
    'num_workers': 2,
    'batch_size': 64,
}

In [16]:
class MyDataset1(Dataset):
    def __init__(self, df, aug, one_hot_transform, train=False, feature_cols=['rel_pos_x',
     'rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
     'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x',
     'rel_acceleration_y', 'rel_acceleration_mag', 'rel_acceleration_ori',
     'G_flug', 'orientation_1', 'orientation_2']):

        self.df = df
        self.features = feature_cols
        self.aug = aug
        self.train = train
        self.one_hot_transform = one_hot_transform

    def __len__(self):
        return ceil(len(self.df)/CFG['batch_size'])

    def get_rows(self, lnum, unum):
        lnum = lnum % len(self.df)
        unum = unum % len(self.df)

        if lnum < unum:
            return self.df[lnum:unum]
        else:
            if self.train:
                return pd.concat([self.df[lnum:], self.df[:unum]])
            else:
                return self.df[lnum:]

    def normalize_features(self, features):
        """
        normalizes the features of the players

       'rel_pos_x',
       'rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
       'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x',
       'rel_acceleration_y', 'rel_acceleration_mag', 'rel_acceleration_ori',
       'G_flug', 'orientation_1', 'orientation_2'
        """
        features /= 100
        features[3] /= 3.6
        features[7] /= 3.6
        features[11] /= 3.6
        features[13] /= 3.6
        features[14] /= 3.6
        return features

    def __getitem__(self, idx):
        window = 24
        frames_to_skip = 4

        row = self.get_rows(
            idx*(CFG['batch_size']), (idx+1)*(CFG['batch_size'])).reset_index(drop=True)
        
        mid_frame = row['frame']
        label = np.array(row['contact']).astype(np.float32)
        args = []
        for i in range(len(row)):
            for view in ['Endzone', 'Sideline']:
                video = row.iloc[i]['game_play'] + f'_{view}.mp4'
                cur_mid_frame = mid_frame.iloc[i]
                frames = [cur_mid_frame - window +
                          i for i in range(0, 2*window+1, frames_to_skip)]
                bbox_col = 'bbox_endzone' if view == 'Endzone' else 'bbox_sideline'
                bboxes = row.iloc[i][bbox_col][::frames_to_skip].astype(
                    np.int32)
#                 print(bboxes)

                if bboxes.sum() <= 0:
                    args += [('dummy', 0, 0, view, self.aug)]*len(frames)
                    continue

                for j, frame in enumerate(frames):
                    cx, cy = bboxes[j]
                    path = f'/kaggle/work/frames/{video}_{frame:04d}.jpg'
                    args.append((path, cx, cy, view, self.aug))
#         imgs = []
#         for arg in args:
#             imgs.append(read_image(*arg))

        if __name__ == '__main__':
            with Pool(CFG['num_workers']) as pool:
                imgs = list(pool.starmap(read_image, args))
                pool.close()
        
        img = torch.stack(imgs).reshape(len(row), 26, 256, 256)
        features = np.array(row[self.features], dtype=np.float32)
        features[np.isnan(features)] = 0

        """
        rel_pos_x                0
        rel_pos_y                1
        rel_pos_mag              2
        rel_pos_ori              3
        rel_speed_x              4
        rel_speed_y              5
        rel_speed_mag            6
        rel_speed_ori            7
        rel_acceleration_x       8
        rel_acceleration_y       9
        rel_acceleration_mag     10
        rel_acceleration_ori     11 
        """
        for i in range(len(row)):
            if row.iloc[i]['G_flug']:
                features[i, 6] = row.iloc[i]['speed_1']
                features[i, 7] = row.iloc[i]['direction_1']
                features[i, 10] = row.iloc[i]['acceleration_1']
                features[i, 11] = row.iloc[i]['direction_1']

                features[i, 4] = row.iloc[i]['speed_1'] * \
                    np.sin(row.iloc[i]['direction_1']*np.pi/180)
                features[i, 5] = row.iloc[i]['speed_1'] * \
                    np.cos(row.iloc[i]['direction_1']*np.pi/180)
                features[i, 8] = row.iloc[i]['acceleration_1'] * \
                    np.sin(row.iloc[i]['direction_1']*np.pi/180)
                features[i, 9] = row.iloc[i]['acceleration_1'] * \
                    np.cos(row.iloc[i]['direction_1']*np.pi/180)

            features[i, :] = self.normalize_features(features[i])

        team_pos = np.array(
            row[['team_1', 'position_1', 'team_2', 'position_2']].fillna('Ground'))
        team_pos = self.one_hot_transform.transform(team_pos).toarray()
        return img, torch.from_numpy(np.hstack((features, team_pos)).astype(np.float32)), torch.as_tensor(label)


In [17]:
video2helmets = {}
test_helmets_new = helmet_data.set_index('video')
for video in tqdm(helmet_data.video.unique()):
    video2helmets[video] = test_helmets_new.loc[video].reset_index(drop=True)

100%|██████████| 4/4 [00:00<00:00, 516.86it/s]


In [18]:
video2frames = {}
test_video_metadata = pd.read_csv("/kaggle/input/nfl-player-contact-detection/test_video_metadata.csv")
for game_play in tqdm(test_video_metadata.game_play.unique()):
    for view in ['Endzone', 'Sideline']:
        video = game_play + f'_{view}.mp4'
        video2frames[video] = max(list(map(lambda x:int(x.split('_')[-1].split('.')[0]), \
                                           glob(f'/kaggle/work/frames/{video}*'))))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/nfl-player-contact-detection/test_video_metadata.csv'

In [19]:
class MyDataset2(Dataset):
    def __init__(self, df, aug=valid_aug, mode='test', transform=one_hot):
        self.df = df
        self.frame = df.frame.values
        feature_cols = ['rel_pos_x','rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
            'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x', 'rel_acceleration_y', 'rel_acceleration_mag',
            'rel_acceleration_ori', 'G_flug', 'orientation_1', 'orientation_2']
        self.feature = df[feature_cols].fillna(0).values
        self.players = df[['nfl_player_id_1','nfl_player_id_2']].values
        self.game_play = df.game_play.values
        self.aug = aug
        self.mode = mode
        self.one_hot_transform = transform
        
    def __len__(self):
        return len(self.df)
    
    # @lru_cache(1024)
    # def read_img(self, path):
    #     return cv2.imread(path, 0)
    def normalize_features(self, features):
        """
        normalizes the features of the players

       'rel_pos_x',
       'rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
       'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x',
       'rel_acceleration_y', 'rel_acceleration_mag', 'rel_acceleration_ori',
       'G_flug', 'orientation_1', 'orientation_2'
        """
        features /= 100
        features[3] /= 3.6
        features[7] /= 3.6
        features[11] /= 3.6
        features[13] /= 3.6
        features[14] /= 3.6
        return features
   
    def __getitem__(self, idx):   
        window = 24
        frame = self.frame[idx]
        
#         if self.mode == 'train':
#             frame = frame + random.randint(-6, 6)

        players = []
        for p in self.players[idx]:
            if p == 'G':
                players.append(p)
            else:
                players.append(int(p))
        
        imgs = []
        for view in ['Endzone', 'Sideline']:
            video = self.game_play[idx] + f'_{view}.mp4'

            tmp = video2helmets[video]
#             tmp = tmp.query('@frame-@window<=frame<=@frame+@window')
            tmp[tmp['frame'].between(frame-window, frame+window)]
            tmp = tmp[tmp.nfl_player_id.isin(players)]#.sort_values(['nfl_player_id', 'frame'])
            tmp_frames = tmp.frame.values
            tmp = tmp.groupby('frame')[['left','width','top','height']].mean()
#0.002s

            bboxes = []
            for f in range(frame-window, frame+window+1, 1):
                if f in tmp_frames:
                    x, w, y, h = tmp.loc[f][['left','width','top','height']]
                    bboxes.append([x, w, y, h])
                else:
                    bboxes.append([np.nan, np.nan, np.nan, np.nan])
            bboxes = pd.DataFrame(bboxes).interpolate(limit_direction='both').values
            bboxes = bboxes[::4]

            if bboxes.sum() > 0:
                flag = 1
            else:
                flag = 0
#0.03s
                    
            for i, f in enumerate(range(frame-window, frame+window+1, 4)):
                img_new = np.zeros((256, 256), dtype=np.float32)

                if flag == 1 and f <= video2frames[video]:
                    img = cv2.imread(f'/kaggle/work/frames/{video}_{f:04d}.jpg', 0)

                    x, w, y, h = bboxes[i]
                    if view == 'Endzone':
                        img = img[int(y+h/2)-76:int(y+h/2)+180,int(x+w/2)-128:int(x+w/2)+128].copy()
                    else:
                        img = img[int(y+h/2)-128:int(y+h/2)+128,int(x+w/2)-128:int(x+w/2)+128].copy()
                    img_new[:img.shape[0], :img.shape[1]] = img
                    
                imgs.append(img_new)
                
        features = np.float32(self.feature[idx])
        if self.df.iloc[i]['G_flug']:
                features[i, 6] = self.df.iloc[i]['speed_1']
                features[i, 7] = self.df.iloc[i]['direction_1']
                features[i, 10] = self.df.iloc[i]['acceleration_1']
                features[i, 11] = self.df.iloc[i]['direction_1']

                features[i, 4] = self.df.iloc[i]['speed_1'] * \
                    np.sin(self.df.iloc[i]['direction_1']*np.pi/180)
                features[i, 5] = self.df.iloc[i]['speed_1'] * \
                    np.cos(self.df.iloc[i]['direction_1']*np.pi/180)
                features[i, 8] = self.df.iloc[i]['acceleration_1'] * \
                    np.sin(self.df.iloc[i]['direction_1']*np.pi/180)
                features[i, 9] = self.df.iloc[i]['acceleration_1'] * \
                    np.cos(self.df.iloc[i]['direction_1']*np.pi/180)

        features = self.normalize_features(features)
        team_pos = np.array(
            self.df.iloc[idx][['team_1', 'position_1', 'team_2', 'position_2']].fillna('Ground'))
        team_pos = self.one_hot_transform.transform([team_pos]).toarray()[0]

        img = np.array(imgs).transpose(1, 2, 0)    
        img = self.aug(image=img)["image"]
        label = np.float32(self.df.contact.values[idx])

        return img, np.hstack((features, team_pos)).astype(np.float32), label

NameError: name 'valid_aug' is not defined